In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

#### 지정도서 크롤링

In [2]:
#접속
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

base_url = "https://library.yonsei.ac.kr/coursereserves/search/list?pn="

In [4]:
#크롤링
#소요 시간: 2m

from tqdm import tqdm

data = []

for i in tqdm(range(1, 138)):
    url = base_url + str(i)
    driver.get(url=url)
    
    table_rows = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')

    for row in table_rows:
        subject = row.find_element(By.CSS_SELECTOR, 'td.subject').text
        title_element = row.find_element(By.CSS_SELECTOR, 'td.title a')
        title = title_element.text
        link = title_element.get_attribute('href')
        
        data.append({
            'subject': subject,
            'title': title,
            'link': link
        })

driver.quit()

100%|██████████| 137/137 [02:01<00:00,  1.12it/s]


In [6]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv('../지정도서.csv')

#### 대출 통계 크롤링

In [16]:
#접속
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

base_url = "https://library.yonsei.ac.kr/statistics/favorLoan?category="

In [17]:
#크롤링
#소요 시간: 1m

books = []    
for m in tqdm(range (100, 1000, 100)):
    url = base_url + str(m)
    driver.get(url=url)
    
    rows = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')
    for row in rows:
        title_element = row.find_element(By.CLASS_NAME, 'title').find_element(By.TAG_NAME, 'a')
        title = title_element.get_attribute('title')
        author = row.find_element(By.CLASS_NAME, 'author').text
        link = title_element.get_attribute('href')
        publisher = row.find_element(By.CLASS_NAME, 'publisher').text
        year = row.find_element(By.CLASS_NAME, 'publisher_year').text
        books.append({
            '주류': m,
            '제목': title,
            '저자': author,
            '링크':link,
            '출판사': publisher,
            '출판 연도': year
        })
driver.quit()

100%|██████████| 9/9 [01:00<00:00,  6.67s/it]


In [18]:
df = pd.DataFrame(books)
df.to_csv('../대출통계.csv')